## Crawling Notebook

This notebook will crawl to the www.fbi.gov website and will download all the crimes statistics of each city in the USA, between the years 2010 - 2019.

The crawler will begin its crawling in the main page of the site and will crawl several page and will export all the relvent data to EXCEL files.
This notebook consist all the functions that will crawl,export, merge and edit the desired files from the FBI website that we will use in this project.

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import os

import time      # for testing use only
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import re
from urllib import request #for download excel

In [27]:
#How many crimes came to the attention of law enforcement in my city in 2019? 

def file_edit(name): 
    direct = '/Users/barbenshabat/Downloads/Datascience/project/' + str(name) + '.xls'
    df = pd.read_excel(direct)
    df.dropna(axis=0, thresh=8, inplace=True)
    col_names = df.iloc[0]#col infirst row insted of the real place
    x=df.iloc[:0]
    for i in x: df.rename(columns={i:col_names[i]}, inplace=True)#swich unnamed in real names
    df.reset_index(inplace=True)
    df.drop([0], inplace=True)#hadle reset index
    df.drop(['index'], axis=1,inplace=True)#hadle reset index
    if ('State' in df.columns):
         df.State.fillna(method='ffill', inplace=True)#filling states
    df.to_excel(str(name)+'_eddited_'+'.xlsx', engine='xlsxwriter')#write to excel
    return df

In [28]:
def load_soup_object(url):
    respons = requests.get(url)
    soup = BeautifulSoup(respons.content, "html.parser")
    return soup


In [29]:
def download_file(file_url, name):
    r = requests.get(file_url, allow_redirects=True)
    name = str(name) + '.xls'
    open(name, 'wb').write(r.content)

In [30]:
def file_edit_macro_table_8():
    table_8_files = []
    for year in range(0,10):
        table_8_files.append('Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_'+str((2010+year))+'_eddited_.xlsx')
    table_8_files
    
    col = []
    path = '/Users/barbenshabat/Downloads/Datascience/project/'
    final_col_names = pd.read_excel(path+table_8_files[0]).columns

    for i in range(0,10):
        df = pd.read_excel(path+table_8_files[i])
        df.dropna(axis=1, inplace=True, thresh=100)
        if i in range (3,7):
            values = {'Rape\n(revised\ndefinition)1' : 0, 'Rape\n(legacy\ndefinition)2' :0}
            df.fillna(value=values, inplace=True)
            sum_rape = df['Rape\n(revised\ndefinition)1']+df['Rape\n(legacy\ndefinition)2']
            df.insert(6, 'Rape', sum_rape)
            df.drop(['Rape\n(revised\ndefinition)1','Rape\n(legacy\ndefinition)2'] ,axis=1, inplace=True)
        for j in range(0,len(final_col_names)): df.rename(columns={df.columns[j]:final_col_names[j]},inplace=True)
        df.rename(columns={'Unnamed: 0':'Year'},inplace=True)
        df['Year']=str(2010+i)
        col.append(df.columns)
        df.to_excel(table_8_files[i], engine='xlsxwriter')#write to excel
    return ('done')

In [31]:
def merge_Table_8():
    file_edit_macro_table_8()
    table_8_files = []
    for year in range(0,10):
        table_8_files.append('Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_'+str((2010+year))+'_eddited_.xlsx')
    dfs_list = []
    path ='/Users/barbenshabat/Downloads/Datascience/project/'

    for f in range(0,10):
        df = pd.read_excel(path+table_8_files[f]) # read each file in the folder 
        dfs_list.append(df) # inserting each df to a list

    main_df = pd.concat(dfs_list,axis=0) # concanicating each df into main df 
    main_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    main_df.to_excel('merge_table_8.xlsx',engine='xlsxwriter')
    return main_df

In [32]:
def Crawling_and_downloadin_table_8_10():   
    #----------------------------------------->Crawling

    FBI_url = f"https://www.fbi.gov" #FBI URL
    respons = requests.get(FBI_url)
    soup = BeautifulSoup(respons.content, "html.parser")

    #----------------------------------------->FBI home page -> Crime Statistics

    mtag=soup.find_all("div",attrs={"class":"footer-nav-group"})
    linksToPages=[t['href'] for t in mtag[len(mtag)-1].find_all("a")]
    #linksToPages[2]#link to crime statistics
    soup=load_soup_object(linksToPages[2])

    #----------------------------------------->Crime Statistics -> UCR Publications page

    mtag=soup.find_all("div",attrs={"class":"movable removable mosaic-tile mosaic-plone.app.standardtiles.rawhtml-tile mosaic-tile-background"})[0]
    soup=load_soup_object(mtag("p")[1].a["href"])

    #----------------------------------------->UCR Publications page -> Eace year in Crime in the United States 

    mtag=soup.find_all("li",attrs={"visualClear"})

    years19_05 = []
    how_mani_crimes19_05=[]
    excel_links19_05=[]

    #Fetting links to all yers pages
    for year in range(0,10):                   
        years19_05.append(mtag[year].a["href"])

    #----------------------------------------->Eace year in Crime in the United States -> Excels pages

    #For each yea getting links to excel page from each year
    for x in range(0,10):      
        soup = load_soup_object(years19_05[x])
        mtag=soup.find_all("div",attrs={"qfind_item"})
        how_mani_crimes19_05.append(mtag[0].a["href"])#How many crimes came to the attention of law enforcement in my city in 2019? 

    #------------------------------------------>Download file Excel files

    #Menually download only for 2018, 2017 need to be download!
    file_edit('Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_2018')
    file_edit('Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_2017')

    for y in range(0,10):
        soup=load_soup_object(how_mani_crimes19_05[y])
        x=soup.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['last'])
        for i in range(0, len(x)):
            if x[i].a.string == 'Download Excel':
                excel_links19_05.append(x[i].a["href"])
                download_file(x[i].a["href"], 'Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_'+str(2019-y))
                df=file_edit('Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_'+str(2019-y))


In [33]:
def file_edit_table_78(file):
    df = pd.read_excel(file)
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, thresh=2, inplace=True)
    col_names = df.iloc[0]#col infirst row insted of the real place
    x=df.iloc[:0]
    for i in x: df.rename(columns={i:col_names[i]}, inplace=True)#swich unnamed in real names
    df.reset_index(inplace=True)
    df.drop([0], inplace=True)#hadle reset index
    df.rename(columns={'index':'Year'},inplace=True)
    df['Year']=2010+int(file[file.find('201')+3])
    if ('State' in df.columns):
        df.State.fillna(method='ffill', inplace=True)#filling states
    df.dropna(axis=1,thresh=4, inplace=True)
    values = {df.columns[4]:0, df.columns[5]:0, df.columns[6]:0}
    df.fillna(value=values, inplace=True)
    total_officers = df[df.columns[4]]+df[df.columns[5]]+df[df.columns[6]]
    df.insert(7, 'Total_Police force', total_officers)
    df.to_excel(file+'_eddited'+'.xlsx', engine='xlsxwriter')#write to excel
    return df

In [34]:
def rename_coln_names(df, col_names):
    for j in range(0,len(col_names)): df.rename(columns={df.columns[j]:col_names[j]},inplace=True)
    return df

In [35]:
def merge_table8_and_table78():
    name78='Table_78_Full-time_Law_Enforcement_Employees_by_State_by_City_201' #.xls_eddited.xlsx
    path78='/Users/barbenshabat/Downloads/Datascience/project/Table_78/'
    name_main8='merge_table_8.xlsx'
    path_main8='/Users/barbenshabat/Downloads/Datascience/project/'
    
    df_main8= pd.read_excel(path_main8+name_main8) #LEFT
    
    for y in np.arange(1): 
        df78=pd.read_excel(path78+name78+str(y)+'.xls_eddited.xlsx') #RIGHT
        result = pd.merge(df78, df_main8, how="outer")#on=["Year", "State", "City"]
    return result
        
        

In [36]:
Crawling_and_downloadin_table_8_10()
main_df=merge_Table_8()


In [37]:
merge_table8_and_table78()

,Unnamed: 0,Year,State,City,Population,Total law enforcement employees,Total officers,Total civilians,Total_Police force,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1
0,1,2010,ALABAMA,Abbeville,2950.0,14.0,7.0,7.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2010,ALABAMA,Adamsville,4796.0,27.0,17.0,10.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2010,ALABAMA,Addison,714.0,3.0,3.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2010,ALABAMA,Alabaster,30781.0,77.0,63.0,14.0,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2010,ALABAMA,Albertville,20588.0,58.0,38.0,20.0,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102826,8100,2019,WYOMING,Sheridan,17895.0,NaN,NaN,NaN,NaN,9.0,0.0,4.0,0.0,5.0,369.0,75.0,278.0,16.0,3.0
102827,8101,2019,WYOMING,Thermopolis5,2830.0,NaN,NaN,NaN,NaN,13.0,0.0,0.0,0.0,13.0,34.0,7.0,22.0,5.0,0.0
102828,8102,2019,WYOMING,Torrington,6709.0,NaN,NaN,NaN,NaN,13.0,0.0,4.0,1.0,8.0,48.0,8.0,40.0,0.0,0.0
102829,8103,2019,WYOMING,Wheatland,3544.0,NaN,NaN,NaN,NaN,7.0,0.0,1.0,0.0,6.0,72.0,24.0,45.0,3.0,0.0
